<img src='../../../frameworks/img/EU-Copernicus-EUM-WEKEO_banner_logo_UNOD_banner.png' align='right' width='100%'></img>

<a href="../../../Index.ipynb"><< Index</a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT <br>
**Authors:** B. Loveday (Innoflair UG / EUMETSAT), Hayley Evers-King (EUMETSAT)

<div class="alert alert-block alert-success">
<h3>Ocean case studies</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>

There are no prerequisite notebooks for this module. <br><br>

Users should refer to the following case study for more contextual information:
- **[Coastal deoxygenation: South African red tides](https://www.eumetsat.int/LINKS!!)**  <<FIX

</div>
<hr>

# Red tides in South African Waters
<font color="#138D75">**UN Ocean Decade Challenge 3: Sustainably feed the global population**</font>

### Data used

| Product Description  | Data Store collection ID|  WEkEO HDA ID | Product Navigator |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|
| Sentinel-3 OLCI level-2 Full resolution | EO:EUM:DAT:0407 | EO:EUM:DAT:SENTINEL-3:OL_2_WFR___ | [link](https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_2_WFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%202%20Data&s=advanced) |

### Learning outcomes

At the end of this notebook you will know how to ;
* extract OLCI spectra over the red tide on South Africa's west coast on February 26th, 2022
* plot OLCI spectra over a given region


### Outline

TODO: REPLACE RGB WITH CHL_NN MINIMAP with lat/lon

This code will replicate figure ??? from this case study; **[Coastal deoxygenation: South African red tides](https://www.eumetsat.int/LINKS!!)**  <<FIX



<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Acquiring OLCI data](#section1)
 2. [Reading OLCI spectral data](#section2)
 3. [Plotting OLCI spectra for specific points](#section3)

<hr>

In [1]:
import shutil
import os
import json
import zipfile
import eumdac
import warnings
import xarray as xr
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import glob
import eumartools

from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure
from bokeh.models import HoverTool, Range1d, LinearAxis
from bokeh.layouts import row

warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Acquiring OLCI data
[Back to top](#TOC_TOP)

</div>

Lets create a directory for our download

In [2]:
# Create a download directory for our OLCI products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

Then we need to specify the collection_id for OLCI Level-2 full resolution products (as specified in the "Data Used" section above), as well as the product that we want to retrieve.

In [ ]:
collection_id = 'EO:EUM:DAT:0407'
product_list = ['S3B_OL_2_WFR____20220226T080620_20220226T080920_20220227T190632_0179_063_092_3420_MAR_O_NT_003.SEN3']

In order to allow us to download data from the Data Store via API, we need to provide our credentials. To do this, we need to create a file called `.eumdac_credentials` in our home directory. For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"consumer_key": "<your_consumer_key>",
"consumer_secret": "<your_consumer_secret>"
}
```

You must replace `<your_consumer_key>` and `<your_consumer_secret>` with the information you extract from https://api.eumetsat.int/api-key/. You will need a [EUMETSAT Earth Observation Portal account](https://eoportal.eumetsat.int/) to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page.

*Note: your key and secret are permanent, so you only need to do this once, but you should take care to never share them*

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

In [ ]:
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

Now we have a token, we can ask the Data Store for our specified product.

In [ ]:
datastore = eumdac.DataStore(token)
selected_product = datastore.get_product(product_id=product_list[0], collection_id=collection_id)

Now we can download this product

In [ ]:
with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
    print(f'Downloading {fsrc.name}')
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {fsrc.name} finished.')

And finally we will unzip the product and tidy up

In [ ]:
with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith(str(selected_product)):
            zip_ref.extract(file, download_dir)
    print(f'Unzipping of product {selected_product} finished.')
os.remove(fdst.name)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Reading OLCI spectral data
[Back to top](#TOC_TOP)

</div>

In [3]:
SAFE_directory = glob.glob(os.path.join(download_dir,'*.SEN3'))[0]

In [4]:
# set up data dictionary
data = {
        'pt1':{
        'band_names': [],
        'band_centres': [],
        'band_widths': [],
        'means': [],
        'uppers': [],
        'lowers': [],
        'errs': []},

        'pt2':{
        'band_names': [],
        'band_centres': [],
        'band_widths': [],
        'means': [],
        'uppers': [],
        'lowers': [],
        'errs': []},

        'pt3':{
        'band_names': [],
        'band_centres': [],
        'band_widths': [],
        'means': [],
        'uppers': [],
        'lowers': [],
        'errs': []}
        }

In [5]:
tree = ET.parse(os.path.join(SAFE_directory,'xfdumanifest.xml'))
root = tree.getroot()
match = {"sentinel3":"http://www.esa.int/safe/sentinel/sentinel-3/1.0"}
data['pt1']["band_names"] = [item.attrib["name"] for item in root.findall('.//sentinel3:band', match)]
data['pt1']["band_centres"] = [float(item.text) for item in root.findall('.//sentinel3:centralWavelength', match)]
data['pt1']["band_widths"] = [float(item.text) for item in root.findall('.//sentinel3:bandwidth', match)]

data['pt2']["band_names"] = data['pt1']["band_names"]
data['pt2']["band_centres"] = data['pt1']["band_centres"]
data['pt2']["band_widths"] = data['pt1']["band_widths"]

data['pt3']["band_names"] = data['pt1']["band_names"]
data['pt3']["band_centres"] = data['pt1']["band_centres"]
data['pt3']["band_widths"] = data['pt1']["band_widths"]

In [6]:
# makes area average
points = {
        'pt1':{
        'lons': [18.20, 18.25, 18.25, 18.20, 18.20],
        'lats': [-32.15, -32.15, -32.10, -32.10, -32.15]},

        'pt2':{
        'lons': [18.00, 18.05, 18.05, 18.00, 18.00],
        'lats': [-32.25, -32.25, -32.20, -32.20, -32.25]},

        'pt3':{
        'lons': [16.50, 16.55, 16.55, 16.50, 16.50],
        'lats': [-31.95, -31.95, -31.90, -31.90, -31.95]}
        }

# for minimap plotting only
grid_reduce = 5

In [7]:
geo_file = os.path.join(SAFE_directory,'geo_coordinates.nc')
geo_vars = xr.open_dataset(geo_file)
lon = geo_vars.longitude.data[::grid_reduce, ::grid_reduce]
lat = geo_vars.latitude.data[::grid_reduce, ::grid_reduce]
geo_vars.close()

In [8]:
RGB_dict = {}
exs = []
eys = []
for point in points:
    # read everything we need from netcdf: open all the files at once
    band_vars = xr.open_mfdataset(glob.glob(os.path.join(SAFE_directory,'Oa*.nc')))
    ex, ey, mask = eumartools.subset_image(lon, lat, points[point]['lons'], points[point]['lats'])
    mask = mask.T

    exs.append(ex)
    eys.append(ey)
    
    for band_var in band_vars:
        print(f"Reading:  {band_var} for {point}")
        var = band_vars[band_var].data[::grid_reduce, ::grid_reduce]*mask
        if "_err" in band_var:
            data[point]["errs"].append(np.array(np.nanmean(var)))
        else:
            data[point]["means"].append(np.array(np.nanmean(var)))
            data[point]["uppers"].append(np.array(np.nanmean(var) + np.nanstd(var)))
            data[point]["lowers"].append(np.array(np.nanmean(var) - np.nanstd(var)))

            # this is just to build our quick minimap
            if "pt1" in point:
                if "Oa08_reflectance" in band_var or "Oa06_reflectance" in band_var or "Oa02_reflectance" in band_var:
                    RGB_dict[band_var] = band_vars[band_var].data[::grid_reduce, ::grid_reduce]

    band_vars.close()

Reading:  Oa01_reflectance for pt1
Reading:  Oa01_reflectance_err for pt1
Reading:  Oa02_reflectance for pt1
Reading:  Oa02_reflectance_err for pt1
Reading:  Oa03_reflectance for pt1
Reading:  Oa03_reflectance_err for pt1
Reading:  Oa04_reflectance for pt1
Reading:  Oa04_reflectance_err for pt1
Reading:  Oa05_reflectance for pt1
Reading:  Oa05_reflectance_err for pt1
Reading:  Oa06_reflectance for pt1
Reading:  Oa06_reflectance_err for pt1
Reading:  Oa07_reflectance for pt1
Reading:  Oa07_reflectance_err for pt1
Reading:  Oa08_reflectance for pt1
Reading:  Oa08_reflectance_err for pt1
Reading:  Oa09_reflectance for pt1
Reading:  Oa09_reflectance_err for pt1
Reading:  Oa10_reflectance for pt1
Reading:  Oa10_reflectance_err for pt1
Reading:  Oa11_reflectance for pt1
Reading:  Oa11_reflectance_err for pt1
Reading:  Oa12_reflectance for pt1
Reading:  Oa12_reflectance_err for pt1
Reading:  Oa16_reflectance for pt1
Reading:  Oa16_reflectance_err for pt1
Reading:  Oa17_reflectance for pt1
Rea

In [9]:
# put everything in Pandas dataframes for convenience
df = []
for point in points:
    data[point]["low_band"]  = [a-b for a,b in zip(data[point]["band_centres"], data[point]["band_widths"])]
    data[point]["high_band"] = [a+b for a,b in zip(data[point]["band_centres"], data[point]["band_widths"])]
    data[point]["low_errs"]  = [a-b for a,b in zip(data[point]["means"], data[point]["errs"])]
    data[point]["high_errs"] = [a+b for a,b in zip(data[point]["means"], data[point]["errs"])]
    df.append(pd.DataFrame(data[point]))

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Plotting OLCI spectra for specific points
[Back to top](#TOC_TOP)

</div>

In [10]:
# build minimap
rgb = np.dstack((RGB_dict["Oa08_reflectance"],
                 RGB_dict["Oa06_reflectance"],
                 RGB_dict["Oa02_reflectance"]))

rgb = eumartools.normalise_image(rgb, unhitch=True)
rgb = eumartools.truncate_image(rgb, min_percentile=1, max_percentile=80)
rgb = eumartools.histogram_image(rgb, nbins=512)

# make image array
img = np.empty((np.shape(rgb)[0], np.shape(rgb)[1]), dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((np.shape(rgb)[0], np.shape(rgb)[1], 4))
for ii in range(np.shape(rgb)[-1]):
    view[:, :, ii] = rgb[:,:,ii]*255
# add alpha
view[:, :, 3] = 255

In [22]:
# create a new plot
hover = HoverTool(tooltips=[("(wavelength, rad/ref)", "($x, $y)")])

# spectra
p = figure(plot_width=1000, plot_height=1000,
           title="OLCI reflectances over selected points",
           y_range=(0, max(df[0]["uppers"])*1.1),
           tools=[hover, 'pan', 'box_zoom', 'save', 'reset'],
           toolbar_location="below")

p.title.align = 'center'
p.title.text_font_size = '14pt'
p.xaxis.axis_label = r"$$ \text{Wavelength } [nm] $$"
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label = r"$$\color{black} \text{reflectance } [sr^{-1}] $$"

for sr, c in zip(df,['red', 'blue', 'black']):
    p.varea(source=sr, x="band_centres", y1="lowers", y2="uppers", alpha=0.5, color=c)
    p.line(source=sr, x="band_centres", y="means", line_width=2, color=c)
    p.circle(source=sr, x="band_centres", y="means", line_color="black", fill_color=c, size=4)
    p.segment(source=sr, x0="low_band", y0="means", x1="high_band", y1="means", line_color="black")
    p.yaxis.axis_label_text_color = c
    p.yaxis.axis_label_text_font_size = "14pt"

# minimap
m = figure(plot_width=500, plot_height=500, toolbar_location="below")
m.image_rgba(image=[np.flipud(img)], x=0, y=0, dw=np.shape(rgb)[1], dh=np.shape(rgb)[0])

m.line(x=exs[0], y=[np.shape(img)[0] - i for i in eys[0]], color="red", line_width=2)
m.line(x=exs[1], y=[np.shape(img)[0] - i for i in eys[1]], color="blue", line_width=2)
m.line(x=exs[2], y=[np.shape(img)[0] - i for i in eys[2]], color="black", line_width=2)

m.axis.visible = False ; m.xgrid.visible = False ; m.ygrid.visible = False

show(row(p, m))

<hr>
<a href="../../../Index.ipynb"><< Index</a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>